In [1]:
import os
import duckdb
from ydata_profiling import ProfileReport

In [2]:
data_source_folder = "/workspaces/valuation/data"
output_filename = "dfp.duckdb"
db_path = os.path.join(data_source_folder, output_filename)
source_table = "gold_dfp_dre_pivoted"

# Create or connect to the DuckDB database
conn = duckdb.connect(database=db_path, read_only=False)

# Read data
df = conn.sql(f"SELECT * FROM {source_table}")

In [3]:
df_clean = df.filter("RECEITA is not Null AND RECEITA_FUTURA is not Null").df()

In [14]:
correlacao = df_clean['RECEITA'].corr(df_clean['RECEITA_FUTURA'])

In [16]:
correlacao

0.47670066536211136

In [15]:
#ProfileReport(df_clean, title="Profiling Report")